# Demystifying async, await, and asyncio


![](https://res.cloudinary.com/practicaldev/image/fetch/s--ClV-OssW--/c_limit%2Cf_auto%2Cfl_progressive%2Cq_auto%2Cw_880/http://maked.io/content/images/2018/06/concurrency-parallelism.jpeg)

## What is synchronous code?

Synchronous code:

- Is what you're used to!
- Runs functions one after another

## What is asynchronous code?

Asynchronous code:

- Runs multiple functions seemingly in parallel
   - In a single process
   - Without threads   
- Requires cooporative, well-behaving functions
   - Functions that regularly suspend by `await`ing something
- Should not use blocking functions!
  - No `time.sleep()`
  - No `socket.*`
  - Etc.
  - `asyncio` provides non-blocking alternatives for many of these functions


## A note on Python versions

- The `async` and `await` keywords were introduced in Python 3.5
- They are syntactic sugar on top of the `asyncio` module that was introduced in Python 3.4
- Python 3.3 and earlier do not support this

- But generator coroutines can do some of the same things

[Documentation](https://docs.python.org/3/library/asyncio.html)<br>
[YouTube](https://www.youtube.com/watch?v=tSLDcRkgTsY)<br>
[BlogPost]()

## Normal Function

In [21]:
import time

def is_prime(x):
    #a = (x//i == x/i for i in range(x-1, 1, -1))
    #print(a)
    return not any(x//i == x/i for i in range(x-1, 1, -1))

def highest_prime_below(x):
    print(f'▶ Highest prime below {x}')
    for y in range(x-1, 0, -1):
        if is_prime(y):
            print(f'▷ Highest prime below {x} is {y}')
            return y
        time.sleep(0.0001)
    return None

def main():
    highest_prime_below(100000)
    highest_prime_below(10000)
    highest_prime_below(1000)
    
main()

▶ Highest prime below 100000
▷ Highest prime below 100000 is 99991
▶ Highest prime below 10000
▷ Highest prime below 10000 is 9973
▶ Highest prime below 1000
▷ Highest prime below 1000 is 997


## Converting to asynchronous.

In [22]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
#https://github.com/spyder-ide/spyder/issues/7096

def is_prime(x):
    return not any(x//i == x/i for i in range(x-1, 1, -1))

async def highest_prime_below(x):
    print(f'▶ Highest prime below {x}')
    for y in range(x-1, 0, -1):
        if is_prime(y):
            print(f'▷ Highest prime below {x} is {y}')
            return y
        await asyncio.sleep(0.01)
        #time.sleep(0.01)
    return None

async def main():
    await asyncio.wait([
        highest_prime_below(100000),
        highest_prime_below(10000),
        highest_prime_below(1000)])
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
#loop.close()

▶ Highest prime below 1000
▶ Highest prime below 100000
▶ Highest prime below 10000
▷ Highest prime below 1000 is 997
▷ Highest prime below 100000 is 99991
▷ Highest prime below 10000 is 9973


## Check the time of taken while using asyncio.sleep() (line16)

In [23]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
#https://github.com/spyder-ide/spyder/issues/7096

def is_prime(x):
    return not any(x//i == x/i for i in range(x-1, 1, -1))

async def highest_prime_below(x):
    print(f'▶ Highest prime below {x}')
    for y in range(x-1, 0, -1):
        if is_prime(y):
            print(f'▷ Highest prime below {x} is {y}')
            return y
        await asyncio.sleep(0.01)
        #time.sleep(0.0001)
    return None

async def main():
    t0 = time.time()
    await asyncio.wait([
        highest_prime_below(100000),
        highest_prime_below(10000),
        highest_prime_below(1000)])
    t1 = time.time()
    print(f'Took {1000*(t1-t0):.2f} ms')
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
#loop.close()

▶ Highest prime below 10000
▶ Highest prime below 100000
▶ Highest prime below 1000
▷ Highest prime below 1000 is 997
▷ Highest prime below 100000 is 99991
▷ Highest prime below 10000 is 9973
Took 606.46 ms


## Check the time of taken while using time.sleep() (line16)

In [24]:
import time
import asyncio
import nest_asyncio
nest_asyncio.apply()
#https://github.com/spyder-ide/spyder/issues/7096

def is_prime(x):
    return not any(x//i == x/i for i in range(x-1, 1, -1))

async def highest_prime_below(x):
    print(f'▶ Highest prime below {x}')
    for y in range(x-1, 0, -1):
        if is_prime(y):
            print(f'▷ Highest prime below {x} is {y}')
            return y
        #await asyncio.sleep(0.01)
        time.sleep(0.01)
    return None

async def main():
    t0 = time.time()
    await asyncio.wait([
        highest_prime_below(100000),
        highest_prime_below(10000),
        highest_prime_below(1000)])
    t1 = time.time()
    print(f'Took {1000*(t1-t0):.2f} ms')
    
loop = asyncio.get_event_loop()
loop.run_until_complete(main())
#loop.close()

▶ Highest prime below 100000
▷ Highest prime below 100000 is 99991
▶ Highest prime below 1000
▷ Highest prime below 1000 is 997
▶ Highest prime below 10000
▷ Highest prime below 10000 is 9973
Took 748.94 ms


### Another example.
[YouTube](https://www.youtube.com/watch?v=BI0asZuqFXM)

In [1]:
%%time
def find_divisible(inrange, div_by):
    print(f"Finding nums in range {inrange} divisible by {div_by}")
    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
    print(f"Done w/ nums in range {inrange} divisible by {div_by}")
    return located

def main():
    divs1 = find_divisible(508000, 34113)
    divs1 = find_divisible(100052, 3210)
    divs1 = find_divisible(500, 3)
    
main()

Finding nums in range 508000 divisible by 34113
Done w/ nums in range 508000 divisible by 34113
Finding nums in range 100052 divisible by 3210
Done w/ nums in range 100052 divisible by 3210
Finding nums in range 500 divisible by 3
Done w/ nums in range 500 divisible by 3
CPU times: user 135 ms, sys: 7.22 ms, total: 142 ms
Wall time: 142 ms


In [3]:
%%time
import asyncio

async def find_divisibles(inrange, div_by):
    print(f"Finding nums in range {inrange} divisible by {div_by}")
    located = []
    for i in range(inrange):
        if i % div_by == 0:
            located.append(i)
        if i % 50000 == 0:
            await asyncio.sleep(0.00001)
    print(f"Done w/ nums in range {inrange} divisible by {div_by}")
    return located

async def main():
    divs1 = loop.create_task(find_divisibles(508000, 34113))
    divs2 = loop.create_task(find_divisibles(100052, 3210))
    divs3 = loop.create_task(find_divisibles(500, 3))
    await asyncio.wait([divs1, divs2, divs3])

try:
    loop = asyncio.get_event_loop()
    loop.run_until_complete(main())
except Exception as e:
    pass
#finally:
#    loop.close()

CPU times: user 213 µs, sys: 29 µs, total: 242 µs
Wall time: 340 µs
Finding nums in range 508000 divisible by 34113
Finding nums in range 100052 divisible by 3210
Finding nums in range 500 divisible by 3
Done w/ nums in range 500 divisible by 3
Done w/ nums in range 100052 divisible by 3210
Done w/ nums in range 508000 divisible by 34113


In [4]:
!pip install nest_asyncio

### async/await in Python 3.5 and why it is awesome

[YouTube](https://www.youtube.com/watch?v=m28fiN9y_r8)

**asyncio is simple, need only seven function to use it.**

- asyncio.get_event_loop()
- loop.create_task()
- loop.run_until_complete() and loop.run_forever()
- asyncio.gather()
- loop.run_in_executor()
- loop.close

**loop.create_task()** <br>
run for ever need CTRL + C to exit.

In [ ]:
async def say(what, when):
    await asyncio.sleep(when)
    print(what)

loop = asyncio.get_event_loop()
loop.create_task(say('hello', 0.5))
loop.create_task(say('world', 1))
loop.run_forever()

**loop.create_task()**

In [30]:
async def say(what, when):
    await asyncio.sleep(when)
    print(what)

t1 = loop.create_task(say('hello', 0.5))
t2 = loop.create_task(say('world', 1))
loop.run_until_complete(asyncio.gather(t1, t2))

hello
world


[None, None]

## Coroutines in Python (async/await)

[YouTube](https://www.youtube.com/watch?v=c6uoxhaenHg)

# asyncio

> To write **concurrent**, **asynchronous**, and **cooperative** code in a sequential style.


### Concurrency vs Parallelism

<img src="https://i.imgur.com/qTBGK9L.png" height=800 width=800>

### Asynchronous vs Synchronous

<img src="https://i.imgur.com/27Hoqxt.png" height=800 width=800>

### Cooperative vs preemptive

<img src="https://i.imgur.com/IBL4310.png" height=800 width=800>

> asyncio is used as a foundation for multiple Python asynchronous frameworks that provide high-performance network and web-servers, database connection libraries, distributed task queues, etc.

## Coroutine

> A function which can pause and resume its execution.

<img src="https://i.imgur.com/ZmHHdzI.png" height=600 width=400>


### How to define a coroutine?

```python
async def main():
    ...
    ...
```

In [33]:
async def main():
    print("Hello")

In [34]:
main()

<coroutine object main at 0x7fb19c18f3b0>

### How to pause execution of coroutine?

```python
async def main():
    await awaitable_object
    ...
    ...
```

> Awaitable objects are: **coroutines**, **Tasks**, and **Futures**.

> **Tasks** are used to schedule coroutines concurrently. When a coroutine is wrapped into a Task with functions like `asyncio.create_task()` the coroutine is automatically scheduled to run soon.

> A **Future** is a special low-level awaitable object that represents an eventual result of an asynchronous operation.
